In [26]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [17]:
df_x = pd.read_csv("../data/concatenated_battery_profile.csv")

In [18]:
df_x.drop(columns='Unnamed: 0', inplace=True)
df_x.head()

,timestamp,current,voltage,temperature
0,1585699584,-0.045,3.990,25.6
1,1585699585,-0.407,3.988,25.6
2,1585699586,-0.925,3.986,25.6
3,1585699587,-2.174,3.981,25.6
4,1585699588,-1.829,3.980,25.6


In [19]:
df_x['t_days'] = (df_x.timestamp-df_x.timestamp[0])/(24*3600)
df_x.head()

,timestamp,current,voltage,temperature,t_days
0,1585699584,-0.045,3.990,25.6,0.000000
1,1585699585,-0.407,3.988,25.6,0.000012
2,1585699586,-0.925,3.986,25.6,0.000023
3,1585699587,-2.174,3.981,25.6,0.000035
4,1585699588,-1.829,3.980,25.6,0.000046


Make a power law model to interpolate synthetic y values for training from the 4 ground truth values we're given. Obviously this is making an assumption about the nature of the fade, but we only have 4 training values, so what're we gonna do

In [20]:
df_y = pd.read_csv("../data/target.csv")
df_y

,timestamp,Q_discharge
0,1.585731e+09,-11.403694
1,1.587712e+09,-11.341753
2,1.590127e+09,-11.307378
3,1.592392e+09,-11.277411


In [77]:
df_y['t_days'] = (df_y.timestamp-df_y.timestamp[0])/(24*3600)
df_y['y'] = np.abs(df_y.Q_discharge)
df_y.head()

,timestamp,Q_discharge,t_days,log_t_days,abs_Q_discharge,y
0,1.585731e+09,-11.403694,0.000000,-inf,11.403694,11.403694
1,1.587712e+09,-11.341753,22.933657,3.132606,11.341753,11.341753
2,1.590127e+09,-11.307378,50.879537,3.929461,11.307378,11.307378
3,1.592392e+09,-11.277411,77.089931,4.344973,11.277411,11.277411


In [78]:
# Define function for calculating a power law, and an error function
capacityFadeModel = lambda q0, t, amp, power: q0 - (amp*t)**power
def error(p, x, y):
    y0 = y[0]
    yPred = capacityFadeModel(y0, x, p[0], p[1])
    return (y-yPred)

In [79]:
(df_y.t_days.values, df_y.y.values)
error([1,1], df_y.t_days.values, df_y.abs_Q_discharge.values)

array([ 0.        , 22.87171653, 50.78322168, 76.96364801])

In [82]:
params, _ = optimize.leastsq(error, [1,1], args=(df_y.t_days.values, df_y.abs_Q_discharge.values))
amp = params[0]
power = params[1]
amp, power

(0.00039428409797607783, 0.5942629179985293)